In [37]:
import sys
import os
import numpy as np
import pandas as pd


In [2]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [3]:
import keras

Using TensorFlow backend.


In [38]:
from keras.models import Sequential

In [39]:

from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras .utils import np_utils

In [40]:
data = pd.read_csv("fer2013.csv")
data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [41]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
emotion    35887 non-null int64
pixels     35887 non-null object
Usage      35887 non-null object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB
None


In [42]:
print(data['Usage'].value_counts())

Training       28709
PublicTest      3589
PrivateTest     3589
Name: Usage, dtype: int64


In [43]:
data.emotion.map({0:0,1:0,2:2,3:3,4:4,5:5,6:6},na_action="ignore")

0        0
1        0
2        2
3        4
4        6
        ..
35882    6
35883    3
35884    0
35885    3
35886    2
Name: emotion, Length: 35887, dtype: int64

In [44]:
x_train, y_train, x_test, y_test = [],[],[],[]

In [45]:
for index, row in data.iterrows():
    val = row["pixels"].split(" ")
    try:
        if 'Training' in row["Usage"]:
            x_train.append(np.array(val, 'float32'))
            y_train.append(row["emotion"])
        elif 'PublicTest' in row["Usage"]:
            x_test.append(np.array(val, 'float32'))
            y_test.append(row["emotion"])
    except:
        print(f"error occured at index:{index} and row:{row}")

In [46]:
print(f"x_train sample data:{x_train[0:2]}")
print(f"y_train sample data:{y_train[0:2]}")
print(f"x_test sample data:{x_test[0:2]}")
print(f"y_test sample data:{y_test[0:2]}")


x_train sample data:[array([ 70.,  80.,  82., ..., 106., 109.,  82.], dtype=float32), array([151., 150., 147., ..., 193., 183., 184.], dtype=float32)]
y_train sample data:[0, 0]
x_test sample data:[array([254., 254., 254., ...,  42., 129., 180.], dtype=float32), array([156., 184., 198., ..., 172., 167., 161.], dtype=float32)]
y_test sample data:[0, 1]


In [47]:
x_train = np.array(x_train,"float32")
y_train = np.array(y_train, "float32")
x_test = np.array(x_test,"float32")
y_test = np.array(y_test,"float32")

In [48]:
# Normalizing the data between 0 and 1

x_train-=np.mean(x_train,axis=0)
x_train/=np.std(x_train,axis=0)

x_test-=np.mean(x_test,axis=0)
x_test/=np.std(x_test,axis=0)


In [49]:
num_features = 64
num_labels = 7
batch_size = 64
epochs = 101 
width, height = 48, 48

In [50]:
x_train = x_train.reshape(x_train.shape[0], width, height,1)
x_test = x_test.reshape(x_test.shape[0],width,height,1)

y_train = np_utils.to_categorical(y_train,num_classes=num_labels)
y_test = np_utils.to_categorical(y_test,num_classes=num_labels)

In [21]:
#desining in cnn

In [51]:
model = Sequential()

In [52]:
#1st layers

model.add(Conv2D(num_features,kernel_size=(3,3),activation="relu",input_shape=(x_train.shape[1:])))
model.add(Conv2D(num_features,kernel_size=(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.5))


In [53]:
#2nd convolutional layers

model.add(Conv2D(num_features,kernel_size=(3,3),activation="relu"))
model.add(Conv2D(num_features,kernel_size=(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.5))

In [54]:
#3rd convolutional layers
model.add(Conv2D(2*num_features,kernel_size=(3,3),activation="relu"))
model.add(Conv2D(2*num_features,kernel_size=(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))


In [55]:
model.add(Flatten())

In [56]:

model.add(Dense(2*2*2*2*num_features,activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(2*2*2*2*num_features,activation = "relu"))
model.add(Dropout(0.2))

model.add(Dense(num_labels, activation = "softmax"))

In [57]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 44, 44, 64)        36928     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 22, 22, 64)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 20, 20, 64)        36928     
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 18, 18, 64)        36928     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 9, 9, 64)          0         
__________

In [58]:
model.compile(loss=categorical_crossentropy,optimizer=Adam(),metrics=['accuracy'])

In [59]:
model.fit(x_train,y_train,batch_size=batch_size,
         epochs = epochs,
         verbose=1,
         validation_data=(x_test,y_test),
         shuffle=True)

W0912 11:14:45.070515 12276 deprecation.py:323] From C:\Users\Shibu\Anaconda3\envs\tensorflow_gpu\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 28709 samples, validate on 3589 samples
Epoch 1/101
28709/28709 [==============================] - 64s 2ms/step - loss: 1.7266 - acc: 0.2941 - val_loss: 1.5581 - val_acc: 0.3714
Epoch 2/101
28709/28709 [==============================] - 49s 2ms/step - loss: 1.5146 - acc: 0.4022 - val_loss: 1.3958 - val_acc: 0.4567
Epoch 3/101
28709/28709 [==============================] - 50s 2ms/step - loss: 1.4082 - acc: 0.4520 - val_loss: 1.3323 - val_acc: 0.4898
Epoch 4/101
28709/28709 [==============================] - 50s 2ms/step - loss: 1.3443 - acc: 0.4796 - val_loss: 1.2709 - val_acc: 0.5116
Epoch 5/101
28709/28709 [==============================] - 50s 2ms/step - loss: 1.2965 - acc: 0.5023 - val_loss: 1.2780 - val_acc: 0.5160
Epoch 6/101
28709/28709 [==============================] - 50s 2ms/step - loss: 1.2658 - acc: 0.5150 - val_loss: 1.2266 - val_acc: 0.5366
Epoch 7/101
28709/28709 [==============================] - 50s 2ms/step - loss: 1.2332 - acc: 0.5252 - val_loss: 1.2002 - v

In [ ]:
fer_json = model.to_json()
with open("fer.json","w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer.h5")    